In [1]:
import os
import shutil
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from sklearn.model_selection import train_test_split

print("All libraries imported successfully!")
print(f"CUDA available: {torch.cuda.is_available()}")

All libraries imported successfully!
CUDA available: False


In [2]:
# Define the 3 tumor types
classes = ['Glioma', 'Meningioma', 'Pituitary']
num_classes = len(classes)
class_to_idx = {'Glioma': 0, 'Meningioma': 1, 'Pituitary': 2}

print("Classes defined:", classes)

# Custom Dataset
class ImageDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

print("Dataset class created!")

Classes defined: ['Glioma', 'Meningioma', 'Pituitary']
Dataset class created!


In [3]:
# Function to get label from filename
def get_label_from_filename(filename):
    name = filename.lower()
    if 'glioma' in name:
        return class_to_idx['Glioma']
    elif 'meningioma' in name:
        return class_to_idx['Meningioma']
    elif 'pituitary' in name:
        return class_to_idx['Pituitary']
    else:
        return None  # Not a tumor type we recognize

# Load images from mri/Tumor folder
mri_tumor_dir = 'mri/Tumor'
image_paths = []
labels = []

print("Scanning MRI tumor images...")

for filename in os.listdir(mri_tumor_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        label = get_label_from_filename(filename)
        if label is not None:
            image_paths.append(os.path.join(mri_tumor_dir, filename))
            labels.append(label)

print(f"Found {len(image_paths)} labeled MRI tumor images")
print(f"Glioma: {labels.count(0)}, Meningioma: {labels.count(1)}, Pituitary: {labels.count(2)}")

Scanning MRI tumor images...
Found 2413 labeled MRI tumor images
Glioma: 672, Meningioma: 1112, Pituitary: 629


In [4]:
# Split into train and validation
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

print(f"Training images: {len(train_paths)}")
print(f"Validation images: {len(val_paths)}")

# Image transformations (same as ImageNet)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets and loaders
train_dataset = ImageDataset(train_paths, train_labels, transform)
val_dataset = ImageDataset(val_paths, val_labels, transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print("Data loaders ready!")

Training images: 1930
Validation images: 483
Data loaders ready!


In [5]:
# Load pre-trained ResNet18
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Change last layer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(f"Model loaded on: {device}")
print("Ready to train!")

c:\Users\vautr\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\vautr\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\vautr/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:28<00:00, 1.62MB/s]


Model loaded on: cpu
Ready to train!


In [6]:
def train_one_epoch():
    model.train()
    running_loss = 0.0
    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if i % 10 == 9:  # Print every 10 batches
            print(f"  Batch {i+1}, Loss: {loss.item():.4f}")
    
    return running_loss / len(train_loader)

def validate():
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    accuracy = 100 * correct / total
    print(f"Validation Accuracy: {accuracy:.2f}%")
    return accuracy

# === Start Training ===
print("Starting training...")
for epoch in range(1, 11):  # 10 epochs
    print(f"\nEpoch {epoch}/10")
    loss = train_one_epoch()
    print(f"Epoch {epoch} - Average Loss: {loss:.4f}")
    validate()

print("Training finished!")

Starting training...

Epoch 1/10
  Batch 10, Loss: 0.6076
  Batch 20, Loss: 0.1758
  Batch 30, Loss: 0.4222
  Batch 40, Loss: 0.4541
  Batch 50, Loss: 0.1162
  Batch 60, Loss: 0.0349
Epoch 1 - Average Loss: 0.3527
Validation Accuracy: 85.30%

Epoch 2/10
  Batch 10, Loss: 0.0834
  Batch 20, Loss: 0.1450
  Batch 30, Loss: 0.0889
  Batch 40, Loss: 0.0435
  Batch 50, Loss: 0.1482
  Batch 60, Loss: 0.0349
Epoch 2 - Average Loss: 0.1011
Validation Accuracy: 96.48%

Epoch 3/10
  Batch 10, Loss: 0.0235
  Batch 20, Loss: 0.0267
  Batch 30, Loss: 0.0387
  Batch 40, Loss: 0.0874
  Batch 50, Loss: 0.2952
  Batch 60, Loss: 0.0191
Epoch 3 - Average Loss: 0.1084
Validation Accuracy: 95.65%

Epoch 4/10
  Batch 10, Loss: 0.1024
  Batch 20, Loss: 0.0228
  Batch 30, Loss: 0.0253
  Batch 40, Loss: 0.0052
  Batch 50, Loss: 0.0301
  Batch 60, Loss: 0.0341
Epoch 4 - Average Loss: 0.0583
Validation Accuracy: 90.06%

Epoch 5/10
  Batch 10, Loss: 0.2999
  Batch 20, Loss: 0.0159
  Batch 30, Loss: 0.2317
  Batch 

In [7]:
# Predict on CT images
ct_tumor_dir = 'ct/Tumor'
ct_images = [os.path.join(ct_tumor_dir, f) for f in os.listdir(ct_tumor_dir) 
             if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

# Create subfolders
for cls in classes:
    os.makedirs(os.path.join(ct_tumor_dir, cls), exist_ok=True)

print(f"Predicting labels for {len(ct_images)} CT images...")

model.eval()
with torch.no_grad():
    for img_path in ct_images:
        image = Image.open(img_path).convert('RGB')
        image = transform(image).unsqueeze(0).to(device)
        
        output = model(image)
        _, predicted = torch.max(output, 1)
        pred_class = classes[predicted.item()]
        
        # Move file
        new_path = os.path.join(ct_tumor_dir, pred_class, os.path.basename(img_path))
        shutil.move(img_path, new_path)
        
        print(f"{os.path.basename(img_path)} → {pred_class}")

print("\nLabeling complete! Check your ct/Tumor/Glioma, Meningioma, Pituitary folders")

Predicting labels for 2318 CT images...
ct_tumor (1).jpg → Meningioma
ct_tumor (1).png → Meningioma
ct_tumor (10).jpg → Meningioma
ct_tumor (10).png → Meningioma
ct_tumor (100).jpg → Meningioma
ct_tumor (100).png → Meningioma
ct_tumor (1000).jpg → Meningioma
ct_tumor (1001).jpg → Meningioma
ct_tumor (1002).jpg → Meningioma
ct_tumor (1003).jpg → Meningioma
ct_tumor (1004).jpg → Meningioma
ct_tumor (1005).jpg → Meningioma
ct_tumor (1006).jpg → Meningioma
ct_tumor (1007).jpg → Meningioma
ct_tumor (1008).jpg → Meningioma
ct_tumor (1009).jpg → Meningioma
ct_tumor (101).jpg → Meningioma
ct_tumor (101).png → Meningioma
ct_tumor (1010).jpg → Meningioma
ct_tumor (1011).jpg → Meningioma
ct_tumor (1012).jpg → Meningioma
ct_tumor (1013).jpg → Meningioma
ct_tumor (1014).jpg → Meningioma
ct_tumor (1015).jpg → Meningioma
ct_tumor (1016).jpg → Meningioma
ct_tumor (1017).jpg → Meningioma
ct_tumor (1018).jpg → Meningioma
ct_tumor (1019).jpg → Meningioma
ct_tumor (102).jpg → Meningioma
ct_tumor (102).png